In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df=pd.read_excel('Book.xlsx')

In [8]:
df.tail()

,Role,Description
1995,General,Adaptable and proficient in learning new conce...
1996,General,Learned and adapted quickly to new technology ...
1997,General,Proved successful working within tight deadlin...
1998,General,Developed and maintained courteous and effecti...
1999,General,Demonstrated strong organizational and time ma...


In [9]:
dataset=df['Description'].tolist()

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(dataset)

/home/parthieshwar/anaconda3/envs/resume/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-11-06 21:55:56.146050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730910356.291652    6270 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730910356.329813    6270 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 21:55:56.567633: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use avai

In [11]:
print(embeddings)

[[ 0.02953494  0.00985249  0.00713057 ...  0.02681286 -0.00196527
   0.00395832]
 [-0.09148374  0.02218495  0.01291202 ...  0.02088753 -0.00189407
  -0.06941591]
 [-0.06537709 -0.03978824  0.00938524 ...  0.0268168   0.03302891
   0.06362893]
 ...
 [-0.01435405  0.05117488  0.05351962 ...  0.01996557 -0.00445964
   0.03013699]
 [-0.05005345  0.05051037  0.00493175 ...  0.00454688  0.06405856
   0.03516557]
 [ 0.004951   -0.01391324  0.00609986 ... -0.00312178 -0.05675127
   0.0220571 ]]


In [12]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(embeddings)

df['cluster'] = clusters

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def find_related_descriptions(user_input, data, model, embeddings):
    exact_matches = data[data['Role'].str.lower() == user_input.lower()]
    user_embedding = model.encode([user_input])
    similarities = cosine_similarity(user_embedding, embeddings).flatten()
    top_indices = np.argsort(similarities)[::-1][:30]
    related_descriptions = data.iloc[top_indices][['Role', 'Description', 'cluster']]
    results = pd.concat([exact_matches, related_descriptions]).drop_duplicates().reset_index(drop=True)
    return results['Description']

user_input = str(input("Role : "))
related_descriptions = find_related_descriptions(user_input, df, model, embeddings)
print(related_descriptions)

0     Handled human resources tasks such as hiring, ...
1     Collaborated with Human Resources Department t...
2     Enhanced employee satisfaction by implementing...
3     Streamlined human resources functions such as ...
4     Managed human resources functions such as recr...
5     Managed complex calendars and travel arrangeme...
6     Assisted human resources team with recruitment...
7     Coordinated human resources initiatives in col...
8     Collaborated with human resources departments ...
9     Maintained up-to-date knowledge of industry be...
10    Partnered with human resources to recruit and ...
11    Managed annual budget for human resources depa...
12    Coordinated closely with human resources to in...
13    Partnered with Human Resources to align talent...
14    Partnered with human resources departments to ...
15    Worked collaboratively with human resources pe...
16    Collaborated with human resources to recruit, ...
17    Enhanced employee productivity by developi